In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from utils.df_loader import load_adult_df, load_compas_df, load_german_df, load_diabetes_df, load_breast_cancer_df
from sklearn.model_selection import train_test_split
from utils.preprocessing import preprocess_df
from utils.models import train_three_models, evaluation_test, save_three_models, load_three_models
from utils.cf_proto import  generate_cf_proto_result, process_result

from utils.save import save_result_as_csv

### Disable TF2 and enable TF1 for alibi.
tf.get_logger().setLevel(40) 
tf.compat.v1.disable_v2_behavior()
tf.keras.backend.clear_session()
pd.options.mode.chained_assignment = None 
tf.compat.v1.disable_eager_execution()
print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False


seed = 123
# tf.random.set_seed(seed)
# np.random.seed(seed)


TF version:  2.0.0
Eager execution enabled:  False


In [2]:
# pip install alibi

In [3]:
#### Select dataset ####

dataset_name = 'compas' # [adult, german, compas]

if dataset_name == 'adult':
    dataset_loading_fn = load_adult_df
elif dataset_name == 'german':
    dataset_loading_fn = load_german_df
elif dataset_name == 'compas':
    dataset_loading_fn = load_compas_df
elif dataset_name == 'diabetes':
    dataset_loading_fn = load_diabetes_df
elif dataset_name == 'breast_cancer':
    dataset_loading_fn = load_breast_cancer_df
else:
    raise Exception("Unsupported dataset")

In [4]:
#### Load datafram info.
df_info = preprocess_df(dataset_loading_fn)

In [5]:
df_info.categorical_cols

['age_cat',
 'sex',
 'race',
 'c_charge_degree',
 'is_recid',
 'is_violent_recid',
 'two_year_recid',
 'class']

In [6]:
df_info.numerical_cols

['age', 'priors_count', 'days_b_screening_arrest', 'length_of_stay']

In [7]:
### Seperate to train and test set.
train_df, test_df = train_test_split(df_info.dummy_df, train_size=.8, random_state=seed, shuffle=True)

In [8]:
### Input queries for generating counterfactuals
df_info.df.iloc[test_df[:20].index]

,age,age_cat,sex,race,priors_count,days_b_screening_arrest,c_charge_degree,is_recid,is_violent_recid,two_year_recid,length_of_stay,class
4855,28,25 - 45,Female,Other,1,0,F,0,0,0,0,Medium-Low
540,22,Less than 25,Male,Caucasian,6,168,F,1,0,1,62,High
4018,44,25 - 45,Male,African-American,0,1,M,0,0,0,2,Medium-Low
2384,29,25 - 45,Male,African-American,0,0,M,0,0,0,0,Medium-Low
2149,29,25 - 45,Male,African-American,0,0,F,0,0,0,0,Medium-Low
5234,24,Less than 25,Male,African-American,2,1,M,0,0,0,3,Medium-Low
3499,24,Less than 25,Male,African-American,0,0,M,1,0,1,1,Medium-Low
5013,32,25 - 45,Male,African-American,0,0,F,0,0,0,1,Medium-Low
5153,25,25 - 45,Male,Hispanic,2,1,F,1,0,1,0,Medium-Low
3648,37,25 - 45,Female,African-American,4,1,M,0,0,0,0,Medium-Low


In [9]:
### Get training and testing array.
X_train = np.array(train_df[df_info.ohe_feature_names])
y_train = np.array(train_df[df_info.target_name])
X_test = np.array(test_df[df_info.ohe_feature_names])
y_test = np.array(test_df[df_info.target_name])

In [10]:
## Train models.
models = train_three_models(X_train, y_train)

## Save models.
save_three_models(models, dataset_name)

Train on 5771 samples
Epoch 1/20


2022-07-11 11:26:08.074992: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-11 11:26:08.075493: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


5771/5771 [==============================] - 0s 81us/sample - loss: 0.6370 - acc: 0.6621
Epoch 2/20
5771/5771 [==============================] - 0s 23us/sample - loss: 0.5255 - acc: 0.7266
Epoch 3/20
5771/5771 [==============================] - 0s 23us/sample - loss: 0.4930 - acc: 0.7543
Epoch 4/20
5771/5771 [==============================] - 0s 22us/sample - loss: 0.4705 - acc: 0.7718
Epoch 5/20
5771/5771 [==============================] - 0s 22us/sample - loss: 0.4614 - acc: 0.7777
Epoch 6/20
5771/5771 [==============================] - 0s 22us/sample - loss: 0.4482 - acc: 0.7895
Epoch 7/20
5771/5771 [==============================] - 0s 22us/sample - loss: 0.4444 - acc: 0.7915
Epoch 8/20
5771/5771 [==============================] - 0s 22us/sample - loss: 0.4425 - acc: 0.7905
Epoch 9/20
5771/5771 [==============================] - 0s 22us/sample - loss: 0.4433 - acc: 0.7919
Epoch 10/20
5771/5771 [==============================] - 0s 22us/sample - loss: 0.4394 - acc: 0.7952
Epoch 11/2

In [11]:
### Load models
models = load_three_models(X_train.shape[-1], dataset_name)

In [12]:
### Print out accuracy on testset.
evaluation_test(models, X_test, y_test)

DT: [0.7332] | RF [0.7852] | NN [0.8094]


# Alibi

## 1. Counterfactual Prototype

In [13]:
from utils.cf_proto import Recorder

In [14]:
### Setting up the CF generating amount.
num_instances = 3
num_cf_per_instance = 1

In [15]:
# Generate CF
results = generate_cf_proto_result(
    df_info,
    train_df,
    models,
    num_instances,
    num_cf_per_instance,
    X_train,
    X_test,
    y_test,
    max_iters=1000,
    models_to_run=['rfc', 'dt', 'nn'],
    output_int=True
)
result_dfs = process_result(results, df_info)


Finding counterfactual for rfc
instance 0
CF 0


No counterfactual found!


CF not found
instance 1
CF 0
Found CF
instance 2
CF 0


No counterfactual found!


CF not found
Finding counterfactual for nn
instance 0
CF 0


No counterfactual found!


CF not found
instance 1
CF 0
Found CF
instance 2
CF 0


No counterfactual found!


CF not found


In [16]:
i = 0
example_input = X_test[i:i+1]

In [17]:
print(Recorder.wrapped_models['nn'].predict(example_input))
print(Recorder.wrapped_models['dt'].predict(example_input))
print(Recorder.wrapped_models['rfc'].predict(example_input))

[[1]]
[[1]]
[[1]]


In [18]:
print(Recorder.wrapped_models['nn'].predict_proba(example_input))
print(Recorder.wrapped_models['dt'].predict_proba(example_input))
print(Recorder.wrapped_models['rfc'].predict_proba(example_input))

[[0 1]]
[[0 1]]
[[0 1]]


In [19]:
# result_dfs['nn']['Found']

In [20]:
#  [Proto] learning_rate_init=0.01
#  [Watcher] learning_rate = 0.1

In [21]:
### Save result as file.
save_result_as_csv("proto", dataset_name, result_dfs)

Result has been saved to ./results/proto_compas
